In [7]:
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import datetime as dt
import time

In [ ]:
# portfolio of stocks
stocks = ['LULU', 'PYPL', 'ADBE', 'FTNT', 'CL', 'ZTS', 'LDOS', 'MNST', 'ADSK', 'MDLZ', 'ISRG', 'MRK', 'NVO', 'UNH', '4528.T', '4568.T', 'BRO', 'DEMANT.CO', 'HRMY', 'HSTM', 'TWLO']

#### Download historical data from yf API

In [ ]:
import yfinance as yf

start_date = "2020-01-01"
end_date = datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker')

# download data to csv
data.to_csv("stocks.csv")

#### Download analysis from csv downloaded data

In [10]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("stocks.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Forward fill missing values for each column
df = df.fillna(method='ffill')

# Convert all values to float
df = df.astype(float)

# Show the result
df.head()

Ticker           LDOS                                                   FTNT  \
Price            Open       High        Low      Close     Volume       Open   
Date                                                                           
2020-01-02  91.102254  91.287343  90.528466  91.268837   793600.0  21.520000   
2020-01-03  90.269364  92.425687  90.149056  92.064758  1031000.0  21.684000   
2020-01-06  91.620539  92.953204  91.324393  92.879166   799500.0  22.122000   
2020-01-07  92.462689  93.258578  92.268336  92.536720   483900.0  22.466000   
2020-01-08  92.536733  93.545488  92.370150  92.425682  1058400.0  22.389999   

Ticker                                                  ...        BRO  \
Price            High        Low      Close     Volume  ...       Open   
Date                                                    ...              
2020-01-02  22.058001  21.476000  21.936001  5541000.0  ...  38.076927   
2020-01-03  22.306000  21.620001  22.242001  6641000.0  ...  37.673299   
2020-01-06  22.472000  21.906000  22.400000  7369000.0  ...  37.894336   
2020-01-07  22.502001  22.214001  22.378000  3796500.0  ...  38.115381   
2020-01-08  22.879999  22.360001  22.733999  6953000.0  ...  38.038494   

Ticker                                                       4568.T  \
Price            High        Low      Close     Volume         Open   
Date                                                                  
2020-01-02  38.124979  37.673286  38.067318  1354300.0          NaN   
2020-01-03  38.173047  37.644465  38.105774   910800.0          NaN   
2020-01-06  38.153821  37.730955  38.076935  1476300.0  2231.429166   
2020-01-07  38.269150  37.961613  37.961613   782400.0  2205.845150   
2020-01-08  38.490187  37.990439  38.240314  1767500.0  2206.468866   

Ticker                                                        
Price              High          Low        Close     Volume  
Date                                                          
2020-01-02          NaN          NaN          NaN        NaN  
2020-01-03          NaN          NaN          NaN        NaN  
2020-01-06  2247.341203  2181.196974  2195.861084  5632800.0  
2020-01-07  2227.373200  2191.805118  2194.925049  5121900.0  
2020-01-08  2220.821051  2164.348772  2191.180908  5244300.0  

[5 rows x 100 columns]

In [11]:
# # Here you can plot the charts for every ticker

# # For every ticker, plot Close
# import matplotlib.pyplot as plt

# for ticker in df.columns.levels[0]:
#     plt.figure(figsize=(12, 6))
#     plt.plot(df[ticker]['Close'], label=ticker)
#     plt.title(f"{ticker} Close Price")
#     plt.xlabel("Date")
#     plt.ylabel("Close Price")
#     plt.legend()
#     plt.show()

#### Plot of Volume Profile for All Stocks

In [12]:
import plotly.io as pio
pio.renderers.default = 'vscode'
import plotly.express as px
import plotly.graph_objects as go

for stock in stocks:
    # Use the stock data from our df
    df_stock = df[stock][['Close', 'Volume']]

    # Convert the stock to datetime if needed
    if not isinstance(df_stock.index, pd.DatetimeIndex):
        df_stock.index = pd.to_datetime(df_stock.index)

    # plot
    hist = go.Histogram(
        y=df_stock['Close'],
        x=df_stock['Volume'],
        nbinsy=150,
        orientation='h',
        name='Volume Profile',
        marker_color='#B0C4DE',
        opacity=0.5,
        xaxis='x2',
    )

    line = go.Scatter(
        x=df_stock.index,
        y=df_stock['Close'],
        mode='lines',
        name='Close',
        marker_color='blue',
        xaxis='x',
        yaxis='y',
    )

    fig = go.Figure([hist, line])

    fig.update_layout(
        xaxis=dict(
            title='Date',
            anchor='y',
            domain=[0, 1],
            showgrid=False,
            side='bottom'
        ),
        xaxis2=dict(
            title='Sum of Volume',
            overlaying='x',
            side='top',
            showgrid=False
        ),
        yaxis=dict(
            title='Close Price',
            showgrid=True
        ),
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        ),
        height=600,
        title=f'Volume Profile & {stock} Close Prices'
    )

    fig.show()